In [1]:
import astropy.io.fits as fits
import numpy as np
import fitsio, os, sys
from astropy.table import Column
from astropy.table import Table
from math import cos,pi,log10
import matplotlib.pylab as plt
import healpy as hp
import pylab as py
from desitarget.geomask import radec_match_to 

path='/uufs/chpc.utah.edu/common/home/astro/dawson/sarahE/eboss/Dec2019/'
plotpath='/uufs/chpc.utah.edu/common/home/astro/dawson/sarahE/eboss/Dec2019/plots/'

#### Binningrp: 30kpc/h - ~200 Mpc/h
powmax=2.5
powmin=-1.5
delpow=0.19
edges=10.**(np.arange(powmin,powmax,delpow))
mids=[]
for i in range(len(edges)-1):
        mids.append((10**((powmin)+((i+1)*(delpow)))+10**((powmin)+(i*(delpow))))/2)
nbins = len(mids)-1
#print(nbins,mids, edges,len(mids))


In [2]:
import glob
ddfiles = glob.glob(path+'/DD_QSO_v7_1_NGC_S/*_DD.fits') 

qso,h = fitsio.read(path+'eBOSS_QSO_clustering_NGC_v7_1_withS_zerosones.fits',header=True)

In [ ]:
sumarr_rp_DD = np.zeros((10000,nbins))
sumarr_rp_wDD = np.zeros((10000,nbins))

%time
for f in ddfiles[0:1]:
    print(f)
    dd,h = fitsio.read(f,header=True)
    
    ind1 = dd['index1']
    ind2 = dd['index2']
    
    
    weights = []
    
#     w1 = qso[ind1]['zerosones']
#     w2 = qso[ind2]['zerosones']
#     for p in range(len(w1)):
#         print('pair ',p)
#         ww1 = list(w1[p].decode())
#         ww2 = list(w2[p].decode())
#         w1w2 = [int(a)*int(b) for a,b in zip(ww2,ww1)]
#         weights.append(w1w2)
    c= 0 
    for p in range(len(ind1)):
        ww1 = list(qso[ind1[p]]['zerosones'].decode())
        ww2 = list(qso[ind2[p]]['zerosones'].decode())
        #try: 
           # weights.append(np.sum(np.asarray(ww1, dtype='int')*np.asarray(ww2, dtype='int')))
           # print(p,weights[p])
        #except:
       #      pass
        
        if (len(ww1)!= len(ww2)):
            c +=1
            print('pair ',p, len(ww1),len(ww2), c)
            
        
        weights.append(np.sum(np.asarray(ww1[0:np.min([len(ww1),len(ww2)])], dtype='int')*np.asarray(ww2[0:np.min([len(ww1),len(ww2)])], dtype='int')))
    
    
    weights = np.asarray(weights)
    s1 =  qso[ind1]['s']
    s2 = qso[ind2]['s']

    d = dd['dist']
    rp = (s1+s2)*d*pi/180./2.
    pii = abs(s1-s2)
    arr_rp = np.zeros((10000,nbins))
    warr_rp = np.zeros((10000,nbins))

    print('start ...')
    
    for j in range(nbins-1):
        wrp  = ((rp < edges[j+1]) & (rp > edges[j]))
        if np.sum(wrp)>0:
            w = weights[wrp]
            a = pii[wrp]
            print(a)
            arr = np.zeros(10000)
            warr = np.zeros(10000)
            
            for k in range(len(a)):
                arr[int(a[k])] += 1
                warr[int(a[k])] += 1*w[k]

            arr_rp[:,j] = arr   
            warr_rp[:,j] = warr    

    sumarr_rp_DD = sumarr_rp_DD + arr_rp   
    sumarr_rp_wDD = sumarr_rp_wDD + warr_rp   
    
    
    
print(sumarr_rp_DD) 

print(sumarr_rp_wDD) 


In [11]:
len(weights),len(dd)

(46241445, 46241445)

In [12]:
sumarr_rp_wDD

array([[       0.,        0.,        0., ...,  6475753., 14740896.,
               0.],
       [       0.,        0.,        0., ...,  6441118., 14550434.,
               0.],
       [       0.,        0.,        0., ...,  6460796., 15162869.,
               0.],
       ...,
       [       0.,        0.,        0., ...,        0.,        0.,
               0.],
       [       0.,        0.,        0., ...,        0.,        0.,
               0.],
       [       0.,        0.,        0., ...,        0.,        0.,
               0.]])

In [13]:
t = Table(dd)

In [14]:
col = Column(weights, name='w1w2')
t.add_column(col,index=0)

t.write(path+'/DD_QSO_v7_1_NGC_S/'+'spherematch.0_DD_weights.fits')